### Importação das bibliotecas necessárias para a raspagem de dados.

In [35]:
# pip install selenium
# pip install webdriver-manager
# pip install BeautifulSoup4

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())
import re
import os

# from selenium.webdriver.chrome.options import Options
# import requests
# options = Options()
# options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
# options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIX_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.
DOWNLOADS = '/home/thiago/Downloads/'
PRE_PROCESSING = '/home/thiago/tcc_ufrj/PRE_PROCESSING'

## Acessando o site Open Street Map e capturando as rotas pendentes e as rotas finalizadas.

In [ ]:
# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

page_content = navegador.page_source
site = BeautifulSoup(page_content, 'html.parser')
routes = site.findAll('tr')

list_pending_routes = []
list_finished_routes = []
list_routes = []

for route in routes:
    if route.find('span', attrs={'class': 'text-danger'}):
        pending_routes = route.find('span', attrs={'class': 'text-danger'})    
        pending_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        # list_pending_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        print (f"{pending_routes.text}: {PREFIX_URL_DOWNLOAD+pending_link_routes['href']}")

    else:         
        finished_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        # list_finished_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        print(f"FINISHED: {PREFIX_URL_DOWNLOAD+finished_link_routes['href']}")
time.sleep(3)
navegador.close()

#len(list_pending_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_finished_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes

## Acessando a telas para efetuar os downloads - Renomeando os arquivo adicionando o nome dos usuários - Movendo para a pasta PRE_PROCESSING

In [ ]:
navegador = webdriver.Chrome(service=servico)
users = []
for list_route in list_routes:
    time.sleep(3)
    url = list_route[0]
    navegador.get(url) #--> Exemplo onde usamos o Selenium somente com o [.get]

    user_page_content = navegador.page_source
    site_user = BeautifulSoup(user_page_content, 'html.parser')

    if any(td.find('span', attrs={'class': 'text-danger'}) for td in site_user):
        tb_user_name = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[4]/td')
        user_name = tb_user_name.text
        user_name = re.sub(r'\s|\.|\(|\)','_',user_name)        
        print(f'Rota PENDENTE: {list_route} Usuário: {user_name}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(user_name)

    else:
        tb_user_name = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[6]/td')
        user_name = tb_user_name.text
        user_name = re.sub(r'\s|\.|\(|\)','_',user_name)
        print(f'Rota FINALIZADA: {list_route} Usuário: {user_name}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(user_name)
time.sleep(3)
navegador.close()

In [ ]:
## Corrigindo o nome dos arquivos conforme seus usuários
files_to_rename = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".crdownload")]
for file_name in files_to_rename:
        new_name = file_name.replace(".crdownload", "")
        os.rename(os.path.join(DOWNLOADS, file_name), os.path.join(DOWNLOADS, new_name))

files_to_rename_gpx = sorted([arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")], reverse=True)
files_to_rename_gpx

for user, file_to_rename_gpx in zip(users, files_to_rename_gpx):
    if os.listdir(DOWNLOADS):
        old_path = os.path.join(DOWNLOADS, file_to_rename_gpx)        
        new_name = f"{file_to_rename_gpx.replace('.gpx', '')}__{user}.gpx"        
        new_path = os.path.join(DOWNLOADS,new_name)        
        #print(new_path)
        os.rename(old_path, new_path)

In [ ]:
# Movendo os arquivos para a pasta PRE_PROCESSING
time.sleep(3)
files_to_move_pre_processing = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")]
for file_to_move_pre_processing in files_to_move_pre_processing:
    src_path = os.path.join(DOWNLOADS, file_to_move_pre_processing)
    dest_path = os.path.join(PRE_PROCESSING, file_to_move_pre_processing)
    try:
        os.rename(src_path,dest_path)
        print(f"Arquivo '{file_to_move_pre_processing}' movido para a pasta {PRE_PROCESSING}.")
    except Exception as e:
        print(f"Erro ao mover o arquivo: '{file_to_move_pre_processing}': {e}.")

## Executando de uma só vez - Entendo que essa execução deve se dar de uma única vez, pois pode haver alguma falha no momento de renomear os arquivos

In [29]:
# pip install selenium
# pip install webdriver-manager
# pip install BeautifulSoup4

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())
import re
import os

# from selenium.webdriver.chrome.options import Options
# import requests
# options = Options()
# options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
# options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.
# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIX_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.
DOWNLOADS = '/home/thiago/Downloads/'
PRE_PROCESSING = '/home/thiago/tcc_ufrj/PRE_PROCESSING'

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

page_content = navegador.page_source
site = BeautifulSoup(page_content, 'html.parser')
routes = site.findAll('tr')

list_pending_routes = []
list_finished_routes = []
list_routes = []

for route in routes:
    if route.find('span', attrs={'class': 'text-danger'}):
        pending_routes = route.find('span', attrs={'class': 'text-danger'})    
        pending_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        # list_pending_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        print (f"{pending_routes.text}: {PREFIX_URL_DOWNLOAD+pending_link_routes['href']}")

    else:         
        finished_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        # list_finished_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        print(f"FINISHED: {PREFIX_URL_DOWNLOAD+finished_link_routes['href']}")
time.sleep(3)
navegador.close()

#len(list_pending_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_finished_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes

## Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
#navegador = webdriver.Chrome(service=servico)


## Baixando os arquivos 
navegador = webdriver.Chrome(service=servico)
users = []
for list_route in list_routes:
    time.sleep(3)
    url = list_route[0]
    navegador.get(url) #--> Exemplo onde usamos o Selenium somente com o [.get]

    user_page_content = navegador.page_source
    site_user = BeautifulSoup(user_page_content, 'html.parser')

    if any(td.find('span', attrs={'class': 'text-danger'}) for td in site_user):
        tb_user_name = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[4]/td')
        user_name = tb_user_name.text
        user_name = re.sub(r'\s|\.|\(|\)','_',user_name)        
        print(f'Rota PENDENTE: {list_route} Usuário: {user_name}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(user_name)

    else:
        tb_user_name = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[6]/td')
        user_name = tb_user_name.text
        user_name = re.sub(r'\s|\.|\(|\)','_',user_name)
        print(f'Rota FINALIZADA: {list_route} Usuário: {user_name}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(user_name)
time.sleep(3)
navegador.close()


## Corrigindo o nome dos arquivos conforme seus usuários
files_to_rename = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".crdownload")]
for file_name in files_to_rename:
        new_name = file_name.replace(".crdownload", "")
        os.rename(os.path.join(DOWNLOADS, file_name), os.path.join(DOWNLOADS, new_name))

files_to_rename_gpx = sorted([arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")], reverse=True)
files_to_rename_gpx

for user, file_to_rename_gpx in zip(users, files_to_rename_gpx):
    if os.listdir(DOWNLOADS):
        old_path = os.path.join(DOWNLOADS, file_to_rename_gpx)        
        new_name = f"{file_to_rename_gpx.replace('.gpx', '')}__{user}.gpx"        
        new_path = os.path.join(DOWNLOADS,new_name)        
        #print(new_path)
        os.rename(old_path, new_path)


# Movendo os arquivos para a pasta PRE_PROCESSING
time.sleep(3)
files_to_move_pre_processing = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")]
for file_to_move_pre_processing in files_to_move_pre_processing:
    src_path = os.path.join(DOWNLOADS, file_to_move_pre_processing)
    dest_path = os.path.join(PRE_PROCESSING, file_to_move_pre_processing)
    try:
        os.rename(src_path,dest_path)
        print(f"Arquivo '{file_to_move_pre_processing}' movido para a pasta {PRE_PROCESSING}.")
    except Exception as e:
        print(f"Erro ao mover o arquivo: '{file_to_move_pre_processing}': {e}.")

PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9690926
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9690925
PENDING: https://www.openstreetmap.org/user/dragonpilot/traces/9690924
FINISHED: https://www.openstreetmap.org/user/dragonpilot/traces/9690922
FINISHED: https://www.openstreetmap.org/user/dragonpilot/traces/9690921
FINISHED: https://www.openstreetmap.org/user/dragonpilot/traces/9690920
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9690919
FINISHED: https://www.openstreetmap.org/user/dragonpilot/traces/9690918
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9690917
FINISHED: https://www.openstreetmap.org/user/dragonpilot/traces/9690915
FINISHED: https://www.openstreetmap.org/user/dragonpilot/traces/9690914
FINISHED: https://www.openstreetmap.org/user/dragonpilot/traces/9690913
FINISHED: https://www.openstreetmap.org/user/Iago/traces/9690912
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9690911
FINISHE

# VERIFICAR A FORMA DE LER OS ARQUIVOS QUE ESTÃO NA PASTA DE DOWNLOAD - CONVERTER PARA CSV OU PARQUET E APÓS ESSA CONVERSÃO LEVAR PARA O MINIO

## Lendo arquivos do Bucket - Transformando os arquivos .gpx em um dataframe e CSV e iniciando com PySpark

In [16]:
# pip install gpxpy
from io import BytesIO
import gpxpy 
import gpxpy.gpx 
import pandas as pd
import xml.etree.ElementTree as ET

In [ ]:
# pip install minio
import os
from minio import Minio
from minio.error import S3Error
BRONZE_LAYER = 'bronze'

minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)
#print(minioclient.list_buckets())
#print("Quantidade de Buckets", len(minioclient.list_buckets()))

In [30]:
files_to_pre_processing = [arquivo for arquivo in os.listdir(PRE_PROCESSING) if arquivo.endswith(".gpx")] #--> Listando todos os arquivos da pasta preprocessamento com extensão .gpx
files_to_pre_processing


['9632227__Routes_from_sunnypilot_2023_08_19-dev__CHEVROLET_BOLT_EUV_2022__.gpx',
 '9690925__Routes_from_sunnypilot_0_9_4_1-release__HYUNDAI_IONIQ_5_2022__.gpx',
 '9690917__Routes_from_sunnypilot_0_9_4_1-release__KIA_EV6_2022__.gpx',
 '9632230__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_SANTA_FE_2019__.gpx',
 '9690906__Routes_from_top_0_9_4_-_2023_07_05__TOYOTA_COROLLA_HYBRID_TSS2_2019__.gpx',
 '9632218__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_SANTA_FE_2019__.gpx',
 '9632216__Routes_from_sunnypilot_0_9_4_1-release__TOYOTA_RAV4_2019__.gpx',
 '9690926__Routes_from_sunnypilot_0_9_4_1-release__HYUNDAI_IONIQ_5_2022__.gpx',
 '9632214__Routes_from_sunnypilot_2023_08_19-dev__SUBARU_IMPREZA_LIMITED_2019__.gpx',
 '9632228__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_SANTA_FE_2019__.gpx',
 '9690922__Routes_from_dragonpilot_2023_07_05__TOYOTA_COROLLA_HYBRID_TSS2_2019__.gpx',
 '9690905__Routes_from_dragonpilot_2023_07_05__TOYOTA_COROLLA_TSS2_2019__.gpx',
 '9632229__Routes_from_sunnypi

In [39]:
files_to_pre_processing = [arquivo for arquivo in os.listdir(PRE_PROCESSING) if arquivo.endswith(".gpx")] #--> Listando todos os arquivos da pasta preprocessamento com extensão .gpx

for file_to_pre_processing in files_to_pre_processing:
    file_path = os.path.join(PRE_PROCESSING, file_to_pre_processing) # --> Crie o caminho completo para o arquivo GPX

    with open(file_path, 'r', encoding='utf-8') as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    info_rota = [] #--> Lista que servirá de apoio para converter o arquivo .gpx em uma lista
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                info_rota.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevacao' : point.elevation,
                    'time_point' : point.time
                })
    file_to_pre_processing_csv = file_to_pre_processing.replace('.gpx','')
    info_rota_df = pd.DataFrame(info_rota)
    info_rota_df.to_csv(f'{PRE_PROCESSING}/{file_to_pre_processing_csv}.csv', index=False)
    os.remove(file_path)    

In [ ]:
## Aqui os arquivos com a extensão .gpx saem da pasta Download para o bucket MinIO
files_to_datalake = [files for files in os.listdir(PRE_PROCESSING) if files.endswith(".gpx")]
for name_file in files_to_datalake:
    local_path = os.path.join(DOWNLOADS, name_file)
    if os.path.isfile(local_path):
        try:
            minioclient.fput_object(BRONZE_LAYER, name_file, local_path)
            print(f"Arquivo {name_file} enviado com sucesso para o bucket.")
            os.remove(local_path) # --> Após o envio bem sucedido para o bucket o arquivo é excluído da pasta download
        except S3Error as e:
            print(f"Erro ao enviar o arquivo: {name_file} -> Erro: {e}")     

In [25]:
with open('PRE_PROCESSING/9632214__Routes_from_sunnypilot_2023_08_19-dev__SUBARU_IMPREZA_LIMITED_2019__.gpx', 'r', encoding='utf-8') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

In [ ]:
gpx_file = '9632229__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_SANTA_FE_2019__.gpx'

In [22]:
gpx_file = str(gpx_file)

In [23]:
type(gpx_file)

str

In [24]:
with open(gpx_file, 'r', encoding='utf-8') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

FileNotFoundError: [Errno 2] No such file or directory: "<_io.TextIOWrapper name='PRE_PROCESSING/9632214__Routes_from_sunnypilot_2023_08_19-dev__SUBARU_IMPREZA_LIMITED_2019__.gpx' mode='r' encoding='utf-8'>"

In [13]:
gpx

GPX(tracks=[GPXTrack(name='2023-08-22T21:58:37.171082', segments=[GPXTrackSegment(points=[...])])])

In [14]:
type(gpx)

gpxpy.gpx.GPX

In [15]:
# Para retornar a quantidade de pontos (lat+lon+ele) em um arquivo arquivo '.gpx' podemos usar get
# Documentação: https://pypi.org/project/gpxpy/
gpx.get_track_points_no()

1444

In [16]:
# Para retornar a faixa de altitude, a fim de se obter extremos de maior elevação e menor elevação no trajeto percorrido - os valores apresentados são em metros acima do nível do mar
gpx.get_elevation_extremes()

MinimumMaximum(minimum=377.068, maximum=387.808)

In [17]:
# Não endendi para que isso serve
gpx.get_uphill_downhill()

UphillDownhill(uphill=5.158500000000004, downhill=15.25849999999997)

In [19]:
# Obtendo o nome do criador da rota - Entretanto no nosso caso quando usamos esse comando temos uma informação generica - Foi necessário obter o nome do criador da rota com o WebScraping
creator = gpx.creator
creator

'sunnypilot https://github.com/sunnypilot/openpilot'

In [20]:
# Para exibir o conteudo do arquivo .gpx em formato xml
print(gpx.to_xml()[:1000])

<?xml version="1.0" encoding="UTF-8"?>
<gpx xmlns="http://www.topografix.com/GPX/1/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/1/1/gpx.xsd" version="1.1" creator="sunnypilot https://github.com/sunnypilot/openpilot">
  <trk>
    <name>2023-08-22T21:58:37.171082</name>
    <trkseg>
      <trkpt lat="47.24594698457771" lon="-119.84832664631423">
        <ele>387.808</ele>
        <time>2023-08-22T21:59:09.999000Z</time>
      </trkpt>
      <trkpt lat="47.24593511169682" lon="-119.84832656552271">
        <ele>387.71500000000003</ele>
        <time>2023-08-22T21:59:10.099000Z</time>
      </trkpt>
      <trkpt lat="47.24592350696483" lon="-119.848326532356">
        <ele>387.629</ele>
        <time>2023-08-22T21:59:10.199000Z</time>
      </trkpt>
      <trkpt lat="47.24591181618821" lon="-119.84832652890437">
        <ele>387.619</ele>
        <time>2023-08-22T21:59:10.299000Z</time>
      </

In [21]:
# É possivel verificar quantas rotas/trilhas nosso arquivo .gpx possui
len(gpx.tracks)

1

In [22]:
# Como no nosso exemplo acima só temos 1 rota/trilha efetuada, podemos acessar por meio de indice no python, 
# Nesse caso se passa o valor [0] para ter uma precisão exata da rota em que estamos trabalhando
gpx.tracks[0]

GPXTrack(name='2023-08-22T21:58:37.171082', segments=[GPXTrackSegment(points=[...])])

In [25]:
# Agora vamos acessar os segmentos da nossa rota/trilha
gpx.tracks[0].segments[0]

GPXTrackSegment(points=[...])

In [26]:
# Agora podemos acessar os pontos de dados individuais dentro do nosso gpx acessando a matriz de pontos. Aqui tambem podemo0s ver o nome das propriedades do arquivo como elevation e time
gpx.tracks[0].segments[0].points[:10]

[GPXTrackPoint(47.24594698457771, -119.84832664631423, elevation=387.808, time=datetime.datetime(2023, 8, 22, 21, 59, 9, 999000, tzinfo=SimpleTZ("Z"))),
 GPXTrackPoint(47.24593511169682, -119.84832656552271, elevation=387.71500000000003, time=datetime.datetime(2023, 8, 22, 21, 59, 10, 99000, tzinfo=SimpleTZ("Z"))),
 GPXTrackPoint(47.24592350696483, -119.848326532356, elevation=387.629, time=datetime.datetime(2023, 8, 22, 21, 59, 10, 199000, tzinfo=SimpleTZ("Z"))),
 GPXTrackPoint(47.24591181618821, -119.84832652890437, elevation=387.619, time=datetime.datetime(2023, 8, 22, 21, 59, 10, 299000, tzinfo=SimpleTZ("Z"))),
 GPXTrackPoint(47.24590031562128, -119.84832646227642, elevation=387.63, time=datetime.datetime(2023, 8, 22, 21, 59, 10, 399000, tzinfo=SimpleTZ("Z"))),
 GPXTrackPoint(47.24588850120415, -119.84832634924405, elevation=387.645, time=datetime.datetime(2023, 8, 22, 21, 59, 10, 499000, tzinfo=SimpleTZ("Z"))),
 GPXTrackPoint(47.24587677038345, -119.84832621704865, elevation=387.6

In [36]:
# Aqui acessamos as tres camadas: Track - Segments e Point e o passamos para um array em forma de dicionário. Note que podemos acessar cada atributo. No meu caso estou pegando latitude, Longitude, Elevação e a hora de cada ponto (entenda-se como ponto Lat+Long+Ele)
info_rota = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            info_rota.append({
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevacao' : point.elevation,
                'time_point' : point.time
            })

In [37]:
# Exibindo somente os 3 primeiros resultados
info_rota[:3]

[{'latitude': 47.24594698457771,
  'longitude': -119.84832664631423,
  'elevacao': 387.808,
  'time_point': datetime.datetime(2023, 8, 22, 21, 59, 9, 999000, tzinfo=SimpleTZ("Z"))},
 {'latitude': 47.24593511169682,
  'longitude': -119.84832656552271,
  'elevacao': 387.71500000000003,
  'time_point': datetime.datetime(2023, 8, 22, 21, 59, 10, 99000, tzinfo=SimpleTZ("Z"))},
 {'latitude': 47.24592350696483,
  'longitude': -119.848326532356,
  'elevacao': 387.629,
  'time_point': datetime.datetime(2023, 8, 22, 21, 59, 10, 199000, tzinfo=SimpleTZ("Z"))}]

In [38]:
type(info_rota)

list

In [31]:
# Aqui podemos transformar essa informação em um dataframe com a biblioteca pandas
info_rota_df = pd.DataFrame(info_rota)
info_rota_df.head()

,latitude,longitude,elevacao,time_point
0,47.245947,-119.848327,387.808,2023-08-22 21:59:09.999000+00:00
1,47.245935,-119.848327,387.715,2023-08-22 21:59:10.099000+00:00
2,47.245924,-119.848327,387.629,2023-08-22 21:59:10.199000+00:00
3,47.245912,-119.848327,387.619,2023-08-22 21:59:10.299000+00:00
4,47.245900,-119.848326,387.630,2023-08-22 21:59:10.399000+00:00


In [59]:
type(info_rota_df)

pandas.core.frame.DataFrame

In [60]:
# Aqui salvamos o pandas dataframe localmente
info_rota_df.to_csv('pandas_info_rota.csv', index=False)

# <creator> contem o nome do cara que está se deslocando
# existe uma <trkpt> acredito q seja um track point
# <name> contem uma da e hora. Acredito que seja a hora do envio do arquivo gpx
# <trkpt> é um bloco com lat - lon - internamente tem um <ele> de elevação e um <time> que acredito ser o horario que o cliente estava no ponto lat+long+ele

In [65]:
# Aqui salvamos o pandas dataframe localmente
info_rota_df.to_parquet('pd_parquet_info_rota.parquet', index=False)

# <creator> contem o nome do cara que está se deslocando
# existe uma <trkpt> acredito q seja um track point
# <name> contem uma da e hora. Acredito que seja a hora do envio do arquivo gpx
# <trkpt> é um bloco com lat - lon - internamente tem um <ele> de elevação e um <time> que acredito ser o horario que o cliente estava no ponto lat+long+ele

In [64]:
# pip install minio
import os
from minio import Minio
from minio.error import S3Error
BRONZE_LAYER = 'bronze'

minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)
#print(minioclient.list_buckets())
#print("Quantidade de Buckets", len(minioclient.list_buckets()))

## Forma de como transformar uma lista em um spark dataframe

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
# Criando as variáveis que serão utilizadas no Spark
appname = 'tcc-project'
master = 'local'

# Criando a sessão Spark
spark = SparkSession.builder\
    .appName(appname)\
    .master(master)\
    .getOrCreate()

In [ ]:
spark

In [55]:
df = spark.createDataFrame(info_rota)

In [58]:
type(df)

pyspark.sql.dataframe.DataFrame

In [57]:
df.show()

+------------------+------------------+-------------------+--------------------+
|          elevacao|          latitude|          longitude|          time_point|
+------------------+------------------+-------------------+--------------------+
|           387.808| 47.24594698457771|-119.84832664631423|2023-08-22 18:59:...|
|387.71500000000003| 47.24593511169682|-119.84832656552271|2023-08-22 18:59:...|
|           387.629| 47.24592350696483|  -119.848326532356|2023-08-22 18:59:...|
|           387.619| 47.24591181618821|-119.84832652890437|2023-08-22 18:59:...|
|            387.63| 47.24590031562128|-119.84832646227642|2023-08-22 18:59:...|
|           387.645| 47.24588850120415|-119.84832634924405|2023-08-22 18:59:...|
|           387.665| 47.24587677038345|-119.84832621704865|2023-08-22 18:59:...|
|           387.721|  47.2458650219579|-119.84832606443189|2023-08-22 18:59:...|
|           387.693| 47.24585325989609|-119.84832593375106|2023-08-22 18:59:...|
|            387.58| 47.2458

In [ ]:
# Coalesce o DataFrame para um único arquivo
df_single_file = df.coalesce(1)
# Salve o DataFrame coalesced como um único arquivo CSV
df_single_file.write.csv(csv_file_path, header=True, mode="overwrite")

# Dessa forma o spark gera um csv particionado
df.write.csv('pyspark_dataframe.csv', header=True, mode="overwrite")

## Daqui para baixo é para mover para o Minio


In [ ]:
# pip install minio
import os
from minio import Minio
from minio.error import S3Error
from io import BytesIO
import gpxpy 
import gpxpy.gpx 
import pandas as pd
from xml.dom import minidom


DOWNLOADS = '/home/thiago/Downloads/'
BRONZE_LAYER = 'bronze'

minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)

In [ ]:
info_rota = []
arquivos_rotas_gpx = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(BRONZE_LAYER) if arquivo_gpx.object_name.endswith(".gpx")]

In [ ]:
for arquivo_rota_gpx in arquivos_rotas_gpx:    
    try:
        blob_rota_gpx = minioclient.get_object(BRONZE_LAYER, arquivo_rota_gpx.object_name)
        rota_gpx = blob_rota_gpx.data
        rota_gpx = rota_gpx.decode("ISO-8859-1")
        try:
            gpx = gpxpy.parse(rota_gpx)
        
            for track in gpx.tracks:
                for segment in track.segments:
                    for point in segment.points:
                        info_rota.append({
                        'latitude': point.latitude,
                        'longitude': point.longitude,
                        'elevacao' : point.elevation,
                        'time_point' : point.time})
    
        except gpxpy.gpx.GPXXMLSyntaxException as e:
            print(f"Erro de sintaxe XML no arquivo {arquivo_rota_gpx.object_name}: {e}")
    except S3Error as e:
        print(f"Erro ao acessar o objeto: {e}")    

In [ ]:
info_rota

In [ ]:
type(rota_gpx)

In [ ]:
# pip install pyspark
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName('tcc-project').getOrCreate()

## Os arquivos que serão movidos da pasta Download para o MINIO_DATA_LAKE precisam estar transformados em Parquet e CSV.

In [ ]:
# pip install minio
import os
from minio import Minio
from minio.error import S3Error
BRONZE_LAYER = 'bronze'

minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)
#print(minioclient.list_buckets())
#print("Quantidade de Buckets", len(minioclient.list_buckets()))

In [ ]:
## Aqui os arquivos com a extensão .gpx saem da pasta Download para o bucket MinIO
files_to_datalake = [files for files in os.listdir(PRE_PROCESSING) if files.endswith(".gpx")]
for name_file in files_to_datalake:
    local_path = os.path.join(DOWNLOADS, name_file)    
    if os.path.isfile(local_path):
        try:
            minioclient.fput_object(BRONZE_LAYER, name_file, local_path)
            print(f"Arquivo {name_file} enviado com sucesso para o bucket.")
            os.remove(local_path) # --> Após o envio bem sucedido para o bucket o arquivo é excluído da pasta download
        except S3Error as e:
            print(f"Erro ao enviar o arquivo: {name_file} -> Erro: {e}")          

## Códigos não mais usados

In [ ]:
## Aqui os arquivos saem da pasta download para a pasta DATALAKE_STAGE

# files_to_move = [files for files in os.listdir(DOWNLOADS) if files.endswith(".gpx")]
# for name_files in files_to_move:
#     origin_path = os.path.join(DOWNLOADS, name_files)
#     destiny_path = os.path.join(DATALAKE_STAGE, name_files)
#     os.rename(origin_path, destiny_path)
#     print(f"Arquivo {name_files} movido para {destiny_path}")

In [ ]:
## Aqui os arquivos saem da pasta DATALAKE_STAGE para o bucket MinIO

# for item in os.listdir(DATALAKE_STAGE):
#     local_path = os.path.join(DATALAKE_STAGE, item)
#     print(local_path)
# 
#     if os.path.isfile(local_path):
#         try:
#             minioclient.fput_object(DATA_LAKE, item, local_path)
#             print(f"Arquivo {item} enviado com sucesso para o bucket.")
#         except S3Error as err:
#             print(f"Erro ao enviar o arquivo {item}: {err}")

In [ ]:
# ## DADOS PARA INSERIR NA TABELA DE CONTROLE USANDO O BEAUTIFULSOAP ## #
# navegador = requests.get('https://www.openstreetmap.org/user/sunnypilot/traces/9255974')
# site = BeautifulSoup(navegador.text, 'html.parser')
# data_trace = site.find('div', attrs={'class': 'content-body'})
# table_trace = data_trace.find('table')
# 
# filename_row = table_trace.find('th', string='Filename:').parent
# owner_row = table_trace.find('th', string='Owner:').parent
# uploaded_row = table_trace.find('th', string='Uploaded:').parent
# 
# filename = filename_row.find('td').text.strip().replace('(download)','').strip()
# owner = owner_row.find('td').text.strip()
# uploaded = uploaded_row.find('td').text.strip()

In [ ]:
# ## ESTE TRECHO TEM A FINALIDADE DE BAIXAR OS ARQUIVOS ## #
#list_finished_routes
#list_pending_routes

# //*[@id="content"]/div[2]/div/table/tbody/tr[6]/td - finalizado
# //*[@id="content"]/div[2]/div/span = PENDENTE
# //*[@id="content"]/div[2]/div/table/tbody/tr[4]/td - pendente

# ## Trecho descontinuado -- Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
# navegador = webdriver.Chrome(service=servico)
# for list_route in list_routes:
#     sleep(3)
#     url = list_route[0]
#     navegador.get(url)  #--> Exemplo onde usamos o Selenium somente com o [.get]
#     navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
# navegador.close()